In [47]:
# paramter cell do not remove!!
nb_parm='datalake|raw/pdf|Birddiversityanddistribution.pdf'
question = ''

In [48]:
import sys
import os

sys.path.append("/home/jovyan/notebooks")
from Framework.module import Utility

In [49]:
## Do the task After this

In [50]:
bucket_name, path_to_file, file_name = nb_parm.split('|')
print("bucket_name:", bucket_name)
print("path_to_file:", path_to_file)
print("file_name:", file_name)

bucket_name: datalake
path_to_file: raw/pdf
file_name: Birddiversityanddistribution.pdf


In [51]:
client = weaviate.connect_to_custom(
    http_host="host.docker.internal",
    http_port=8081,
    http_secure=False,
    grpc_host="host.docker.internal",
    grpc_port=50051,
    grpc_secure=False,
)
client.collections.delete("BridsKnowledge")

In [52]:
import uuid
import weaviate
import pdfplumber
from weaviate.classes.config import Configure
from datetime import datetime
from weaviate.classes.config import Property, DataType
import io

OLLAMA_API = "http://host.docker.internal:11434"
EMBED_MODEL = "mxbai-embed-large"
GEN_MODEL = "deepseek-r1:7b"

def create_collection(client):
    if "BridsKnowledge" in client.collections.list_all():
        return client.collections.get("BridsKnowledge")

    return client.collections.create(
        name="BridsKnowledge",
        vectorizer_config=Configure.Vectorizer.text2vec_ollama(
            api_endpoint=OLLAMA_API,
            model=EMBED_MODEL
        ),
        generative_config=Configure.Generative.ollama(
            api_endpoint=OLLAMA_API,
            model=GEN_MODEL
        ),
        properties=[
            Property(name="title", data_type=DataType.TEXT),
            Property(name="content", data_type=DataType.TEXT),
            Property(name="source_type", data_type=DataType.TEXT),
            Property(name="source_id", data_type=DataType.TEXT),
            Property(name="chunk_index", data_type=DataType.INT),
            Property(name="timestamp", data_type=DataType.DATE)
        ]
    )

def chunk_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

def ingest_pdf_to_weaviate(bucket_name, path_to_file, title=None):
    client = weaviate.connect_to_custom(
        http_host="host.docker.internal",
        http_port=8081,
        http_secure=False,
        grpc_host="host.docker.internal",
        grpc_port=50051,
        grpc_secure=False,
    )

    kb = create_collection(client)
    
    # Get PDF file as bytes from MinIO
    file_bytes = Utility.af_read_csv(bucket_name, path_to_file)
    
    # title = title or path_to_file.split("/")[-1]
    title = 'pdf-ingest'
    source_id = str(uuid.uuid4())
    timestamp = datetime.utcnow().replace(microsecond=0).isoformat() + "Z"
    
    chunk_index = 0
    with pdfplumber.open(io.BytesIO(file_bytes)) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                chunks = chunk_text(text)
                for chunk in chunks:
                    kb.data.insert({
                        "title": title,
                        "content": chunk,
                        "source_type": "pdf",
                        "source_id": source_id,
                        "chunk_index": chunk_index,
                        "timestamp": timestamp
                    })
                    chunk_index += 1

    client.close()
    print(f"Ingested {chunk_index} chunks from '{title}'")


In [53]:
ingest_pdf_to_weaviate(bucket_name, path_to_file+'/'+file_name)


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Ingested 19 chunks from 'pdf-ingest'


In [54]:
import weaviate
import json
from datetime import datetime
from weaviate.classes.query import MetadataQuery, Filter

def serialize(obj):
    """Helper function to handle serialization of datetime objects."""
    if isinstance(obj, datetime):
        return obj.isoformat()  # Convert datetime to string in ISO format
    raise TypeError("Type not serializable")

client = weaviate.connect_to_custom(
    http_host="host.docker.internal",
    http_port=8081,
    http_secure=False,  # Set to True if using HTTPS
    grpc_host="host.docker.internal",
    grpc_port=50051,
    grpc_secure=False,  # Set to True if using a secure gRPC connection
)

questions = client.collections.get("BridsKnowledge")

# Perform the text generation
response = questions.query.bm25(
    query="conclusion",
    limit=10,
    query_properties=["content"],
    return_metadata=MetadataQuery(score=True),
)

# # Extract and print the text results
# for obj in response.objects:
#     # Convert the properties to JSON string format, handling datetime objects
#     print(json.dumps(obj.properties, default=serialize, indent=2))

#     # # If you are specifically looking for the generated text (depending on the structure)
#     # # assuming the text is stored under a specific property like 'content'
#     # generated_text = obj.properties.get("content", "No content found")
#     # print("Generated Text:", generated_text)

for obj in response.objects:
    print(json.dumps(obj.properties, default=serialize, indent=2))
    print(json.dumps(obj.metadata.score, default=serialize, indent=2))


client.close()  # Free up resources


{
  "title": "pdf-ingest",
  "source_id": "38cd45c9-a99c-4a57-9be1-f3922813de45",
  "chunk_index": 13,
  "content": "generalization were observed. For example Myophonus caeruleus (Scopoli, 1786), Phalacro coraxniger (Vieillot 1817), Rhyacornis fuliginosus, Pycnonotuscafer, Enicurus schistaceus (Hodgson 1837), White-crested Laughingthrush and Phoenicurus erythrogaster (G\u00fcldenst\u00e4dt 1775) were recorded in all habitat type. On the contrary, Passer domesticus (Linnaeus 1758), Aethopyga siparaja (Raffles 1822), Acridothere stristis (Linnaeus 1766), Columba livia (Gmelin 1789), Dicrurus macrocercus (Vieillot 1817), Pycnonotus melanicterus (Gmelin 1789) and Psittacula eupatriawere recorded in settlement. Although human settlement areas have mixtures of built habitats and green patches, bird species have managed to exist and thrive in this complex habitat (Sandstorm et al.2005). This is explained by the availability of ecological requirements for the species offered by a mixture of an